# Enedis 13, import, clean, ban formated

In [345]:
# import libraries
import pandas as pd
import numpy as np
import os
# chemin du dataset deja filtré sur le 13
DATA_PATH = ''
DATA_CSV = 'enedis_13.csv'
# import du csv pour l'exploration
df = pd.read_csv(os.path.join(DATA_PATH, DATA_CSV))
# Pour générer un dictionnaire de base à modifier
{k : 'float32' for k in df.columns}

{'annee': 'float32',
 'code_iris': 'float32',
 'nom_iris': 'float32',
 'numero_de_voie': 'float32',
 'indice_de_repetition': 'float32',
 'type_de_voie': 'float32',
 'libelle_de_voie': 'float32',
 'code_commune': 'float32',
 'nom_commune': 'float32',
 'segment_de_client': 'float32',
 'nombre_de_logements': 'float32',
 'consommation_annuelle_totale_de_l_adresse_mwh': 'float32',
 'consommation_annuelle_moyenne_par_site_de_l_adresse_mwh': 'float32',
 'consommation_annuelle_moyenne_de_la_commune_mwh': 'float32',
 'adresse': 'float32',
 'tri_des_adresses': 'float32',
 'code_epci': 'float32',
 'code_departement': 'float32',
 'code_region': 'float32'}

In [346]:
# convertir le type des columns
d = {
    'annee': 'int64', # de 2018 a 2022
    #'code_iris': 'str', # inutile?
    #'nom_iris': 'str', # inutile?
    #'numero_de_voie': 'str', # redondance
    #'indice_de_repetition': 'str', # drop indice_de_repetition: 56080 / 57678
    #'type_de_voie': 'str', # redondance
    #'libelle_de_voie': 'str', # redondance
    'code_commune': 'str',
    'nom_commune': 'category',
    #'segment_de_client': 'category', # inutile ils sont tous RESIDENTIEL
    'nombre_de_logements': 'int64',
    'consommation_annuelle_totale_de_l_adresse_mwh': 'float64',
    'consommation_annuelle_moyenne_par_site_de_l_adresse_mwh': 'float64',
    'consommation_annuelle_moyenne_de_la_commune_mwh': 'float64',
    'adresse': 'str',
    #'tri_des_adresses': 'int64', # inutile?
    #'code_epci': 'int64', # inutile?
    #'code_departement': 'int64', # redondance
    #'code_region': 'int64'  # inutile?
 }
df = pd.read_csv(os.path.join(DATA_PATH, DATA_CSV), usecols=d.keys(), dtype=d)

In [347]:
# verification des null
df.isna().sum()

annee                                                      0
code_commune                                               0
nom_commune                                                0
nombre_de_logements                                        0
consommation_annuelle_totale_de_l_adresse_mwh              0
consommation_annuelle_moyenne_par_site_de_l_adresse_mwh    0
consommation_annuelle_moyenne_de_la_commune_mwh            0
adresse                                                    0
dtype: int64

In [348]:
# to_csv du pres clean
df.to_csv('enedis_13_clean.csv', index=False)

In [349]:
# load the clean enedis
df_clean = pd.read_csv('enedis_13_clean.csv')

In [350]:
# fonction pour l'utilisation de l'api du gouv pour identifier identifiant ban par rapport a l'adresse, nom et code commune
import requests
from io import BytesIO

input_filepath = 'enedis_13_clean.csv'
url_csv = 'https://api-adresse.data.gouv.fr/search/csv/'
data = {'result_columns': ['result_id', 'result_score'],
        'columns': ['adresse', 'nom_commune'],
        'citycode': 'code_commune'}

def get_ban_id_from_csv(input_filepath,
                        output_filepath='enedis_13_clean_ban_formated.csv',
                        url=url_csv,
                        data=data):

    with open(input_filepath, "rb") as csv_file:
        files = {'data': csv_file}
        r = requests.post(url, files=files, data=data)
        pd.read_csv(BytesIO(r.content)).to_csv(output_filepath, index=False)

In [351]:
# appel de la fonction
%%time
get_ban_id_from_csv('enedis_13_clean.csv')

CPU times: user 1.78 s, sys: 213 ms, total: 1.99 s
Wall time: 3min 4s


In [352]:
df_clean_ban_formated = pd.read_csv('enedis_13_clean_ban_formated.csv')

In [353]:
# le test du status: resolution impossible
# df_final['result_status'].value_counts()#.isna().sum()
result_status = "Correspondance Identifiant BAN:\nok: 57641\nnot-found: 34\nerror: 3"

In [354]:
# filtre de la fiabilitée de la correspondance
# definis le threshold value de correspondance
threshold = 0.75

# filtrer le DataFrame base sur le threshold
df_clean_ban_formated = df_clean_ban_formated[df_clean_ban_formated['result_score'] >= threshold]
#df_final['result_id'].count()

In [355]:
# Compte rendu
print("Nombre de ligne du dataset original sur le dep13:\n57678\n")
print(result_status)
print(f"\nConcordance avec {threshold} de score:\n{df_clean_ban_formated['result_id'].count()}")
print(f"\nNombre d'adresse unique:\n{df_clean_ban_formated['result_id'].nunique()}")
print(f"\nAdresses perdues:\n{57678 - df_clean_ban_formated['result_id'].count()} / 57678")

Nombre de ligne du dataset original sur le dep13:
57678

Correspondance Identifiant BAN:
ok: 57641
not-found: 34
error: 3

Concordance avec 0.75 de score:
55304

Nombre d'adresse unique:
12477

Adresses perdues:
2374 / 57678


In [356]:
# nettoyage final
# drop column code_commune, nom_commune, adresse et result_score (plus utile, servaient pour recuperer l'identifiant BAN et result score)
co = ['code_commune', 'nom_commune', 'adresse', 'result_score']
df_clean_ban_formated.drop(columns=co, inplace=True)

In [357]:
# renommage des columns
df_clean_ban_formated.rename(columns={'result_id': 'Identifiant__BAN'}, inplace=True)

In [359]:
# export enedis 13 final
df_clean_ban_formated.to_csv('enedis_13_final.csv', index=False)

In [360]:
df_clean_ban_formated

,annee,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,Identifiant__BAN
0,2019,16,29.121,1.820,7.149,13085_0241_00004
1,2019,10,69.518,6.952,7.217,13086_0914_00352
2,2019,19,143.778,7.567,7.217,13086_0911_00320
3,2019,22,167.261,7.603,7.322,13076_0025_00594
4,2019,13,51.682,3.976,5.243,13075_3710_00005
...,...,...,...,...,...,...
57673,2021,43,27.335,0.636,4.361,13001_3080_00002
57674,2021,10,15.843,1.584,4.361,13001_3117
57675,2021,42,225.278,5.364,7.077,13002_1770_00009
57676,2021,47,93.977,2.000,7.077,13002_1340_00314


# Resource code (au cas ou)

In [336]:
# nombre d'adesse unique
#df_final['result_id'].nunique()

In [283]:
# selection result au cas ou on doit recuperer de nouvelles columns dans la fonction
'''
df_final[[ 'result_label', 'result_score',
       'result_score_next', 'result_type', 'result_id', 'result_housenumber',
       'result_name', 'result_street', 'result_postcode', 'result_city',
       'result_context', 'result_citycode', 'result_oldcitycode',
       'result_oldcity', 'result_district', 'result_status']].head(3)
'''

"\ndf_final[[ 'result_label', 'result_score',\n       'result_score_next', 'result_type', 'result_id', 'result_housenumber',\n       'result_name', 'result_street', 'result_postcode', 'result_city',\n       'result_context', 'result_citycode', 'result_oldcitycode',\n       'result_oldcity', 'result_district', 'result_status']].head(3)\n"